In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import time
import os

from sentence_transformers import SentenceTransformer

C:\Users\Sikma\AppData\Roaming\Python\Python312\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Scrapping danych i zapis

## Utworzenie dataframe i słownika dat
utworzenie pustego dataframe politicians_df z potrzebnymi kolumnami i stworzenie słownika ze wszystkimi datami i posiedzeniami sejmu

In [2]:
politicians_df = pd.DataFrame(columns=["politician", "political_club", "utterance", "sejm_session", "date"])

In [3]:
gov_meetings_dict = {
    "2023-11-13": [1],
    "2023-11-14": [1],
    "2023-11-21": [1],
    "2023-11-22": [1],
    "2023-11-28": [1],
    "2023-11-29": [1],
    "2023-12-06": [1],
    "2023-12-07": [1],
    "2023-12-11": [1],
    "2023-12-12": [1],
    "2023-12-19": [1],
    "2023-12-20": [1],
    "2023-12-21": [1],
    "2024-01-16": [2],
    "2024-01-17": [2],
    "2024-01-18": [3],
    "2024-01-25": [4],
    "2024-01-26": [4],
    "2024-02-07": [5],
    "2024-02-08": [5],
    "2024-02-09": [5],
    "2024-02-21": [6],
    "2024-02-22": [6],
    "2024-03-06": [7],
    "2024-03-07": [7],
    "2024-03-08": [7],
    "2024-03-20": [8],
    "2024-03-21": [8],
    "2024-04-10": [9],
    "2024-04-11": [9],
    "2024-04-12": [9],
    "2024-04-24": [10],
    "2024-04-25": [10],
    "2024-04-26": [10],
    "2024-05-08": [11],
    "2024-05-09": [11],
    "2024-05-15": [11],
    "2024-05-22": [12],
    "2024-05-23": [12],
    "2024-06-12": [13],
    "2024-06-13": [13],
    "2024-06-14": [13],
    "2024-06-26": [14],
    "2024-06-27": [14],
    "2024-06-28": [14],
    "2024-07-11": [15],
    "2024-07-12": [15],
    "2024-07-23": [16],
    "2024-07-24": [16],
    "2024-07-25": [16],
    "2024-07-26": [16],
    "2024-09-11": [17],
    "2024-09-12": [17],
    "2024-09-13": [17],
    "2024-09-25": [18],
    "2024-09-26": [18],
    "2024-09-27": [18],
    "2024-10-01": [18],
    "2024-10-09": [19],
    "2024-10-10": [19],
    "2024-10-11": [19],
    "2024-10-16": [20],
    "2024-10-17": [20],
    "2024-10-18": [20],
    "2024-11-06": [21],
    "2024-11-07": [21],
    "2024-11-08": [21],
    "2024-11-19": [22],
    "2024-11-20": [22],
    "2024-11-21": [22],
    "2024-11-22": [22],
    "2024-11-27": [22],
    "2024-12-04": [23],
    "2024-12-05": [23],
    "2024-12-06": [24],
    "2024-12-18": [25],
    "2024-12-19": [25],
    "2024-12-20": [25],
    "2025-01-08": [26],
    "2025-01-09": [26],
    "2025-01-10": [26],
    "2025-01-22": [27],
    "2025-01-23": [27],
    "2025-01-24": [27]
}


## Scrapping danych przy użyciu sejmowego API
- przeiterowanie po każdej dacie i numerze posiedzenia sejmu,
- stworzenie URL do API sejmu zawierającego konkretną datę oraz posiedzenie i pobranie każdej wypowiedzi z danego zebrania
- wyjęcie z 'h2' polityka (politician)
- wyjęcie zmiędzy paragrafów 'p' wypowiedzi (utterance)
- zapisanie wszystkiego do politicians_df

In [4]:
for name, key in gov_meetings_dict.items():
    for number in key:
        i = 1
        start_time = time.time()
        while True:
            url = f"https://api.sejm.gov.pl/sejm/term10/proceedings/{number}/{name}/transcripts/{i}"
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html')
        
            element = soup.find('h2', class_='mowca')
            if element is not None:
                politician = element.text.strip()[:-1]
            else:
                print(f'Koniec wypowiedzi. Odnotowano: {i} przemówień')
                break
        
            utterance_elements = []
            for p in soup.find_all('p')[1:]:
                utterance_elements.append(p.text.strip())
            utterance = "".join(utterance_elements).strip()
        
            nowy_wiersz = pd.DataFrame({"politician": [politician], 
                                        "utterance": [utterance], 
                                        "sejm_session": [number], 
                                        "date": [name]})
            politicians_df = pd.concat([politicians_df, nowy_wiersz], ignore_index=True)
        
            i = i+1
        end_time = time.time()
        print(f'Czas posiedzenia nr {number} o dacie {name}: {end_time-start_time:.2f} sekundy \n')

Koniec wypowiedzi. Odnotowano: 43 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-13: 26.31 sekundy 

Koniec wypowiedzi. Odnotowano: 18 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-14: 10.34 sekundy 

Koniec wypowiedzi. Odnotowano: 33 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-21: 18.40 sekundy 

Koniec wypowiedzi. Odnotowano: 116 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-22: 67.61 sekundy 

Koniec wypowiedzi. Odnotowano: 191 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-28: 105.04 sekundy 

Koniec wypowiedzi. Odnotowano: 238 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-29: 136.37 sekundy 

Koniec wypowiedzi. Odnotowano: 156 przemówień
Czas posiedzenia nr 1 o dacie 2023-12-06: 85.88 sekundy 

Koniec wypowiedzi. Odnotowano: 185 przemówień
Czas posiedzenia nr 1 o dacie 2023-12-07: 106.51 sekundy 

Koniec wypowiedzi. Odnotowano: 161 przemówień
Czas posiedzenia nr 1 o dacie 2023-12-11: 89.25 sekundy 

Koniec wypowiedzi. Odnotowano: 268 przemówień
Czas posiedzenia n

## Sprawdzenie poprawności dataframe
różne informacje: pierwsza wartość, próbka, shape itp.

In [5]:
politicians_df.iloc[0,0]

'Prezydent Rzeczypospolitej Polskiej Andrzej Duda'

In [6]:
politicians_df.sample().iloc[0,1]

'Szanowna Pani Marszałek! Szanowna Izbo! Zanim zostałem posłem, byłem radnym \r\nmiasta Częstochowy. To jest pierwsze miasto w Polsce, które wprowadziło in \r\nvitro w 2012 r. W 2015 r. był taki moment, że radni Częstochowy chcieli \r\nzaktualizować ten program, zwiększyć kwotę przyznawaną parom i klub Lewicy i \r\nPlatformy był za, a klub PiS-u był przeciw.W 2015 r. to głosy naszego częstochowskiego klubu, klubu mieszkańców \r\nCzęstochowy, zdecydowały o tym, że do tej pory w Częstochowie funkcjonuje in \r\nvitro i urodziło się 77 dzieci.Cieszę się, że historia zatacza koło, bo wtedy w Częstochowie mieliśmy duży \r\ndylemat przez to, że mieszkańcy, osoby zameldowane w Częstochowie mogą \r\nkorzystać z tego programu, a mieszkańcy sąsiednich gmin (Dzwonek) jak \r\nBlachownia, Koniecpol czy Lubliniec - nie. Cieszę się, że mogę to dzisiaj \r\nnaprawić i razem z państwem zagłosować za in vitro. Dziękuję bardzo. \r\n(Oklaski)'

In [8]:
politicians_df.shape

(15878, 4)

In [9]:
politicians_df.groupby('politician').size().sort_values(ascending=False)

politician
Poseł Witold Tumanowicz                                                                                                       360
Poseł Jarosław Sachajko                                                                                                       322
Poseł Zbigniew Bogucki                                                                                                        205
Poseł Tadeusz Tomaszewski                                                                                                     199
Poseł Grzegorz Braun                                                                                                          182
                                                                                                                             ... 
Poseł Zbigniew Chmielowiec (tekst niewygłoszony)                                                                                1
Poseł Sprawozdawca Alicja Chybicka                                             

## Dodanie danych o partiach politycznych
wykorzystanie sejmowego API do stworzenia słownika zawierającego partie polityczne i ich posłów, a następnie przeiterowanie po datasecie politicians_df aby nadać politykom ich kluby

In [8]:
url_politicians = "https://api.sejm.gov.pl/sejm/term10/MP"
page_politicians = requests.get(url_politicians)
political_parties_data = page_politicians.json()

In [11]:
club_dict = {}

for person in data:
    club = person.get("club", "Brak klubu")  # Pobieramy klub, jeśli brak to "Brak klubu"
    name = person.get("firstLastName", "Nieznany")  # Pobieramy imię i nazwisko

    # Dodajemy do słownika
    if club in club_dict:
        club_dict[club].append(name)
    else:
        club_dict[club] = [name]

#for club, names in club_dict.items():
#    print(f"{club}: {', '.join(names)}")

In [13]:
politicians_df["political_club"] = ""

In [22]:
def assign_political_club(row):
    politician_name = row["politician"]  # Pełna nazwa polityka z tytułem

    for club, names in club_dict.items():
        if any(name in politician_name for name in names):  # Szukamy nazwiska w całym stringu
            return club
    return "Nieznana partia"

politicians_df["political_club"] = politicians_df.apply(assign_political_club, axis=1)

In [35]:
politicians_df.sample()

,politician,utterance,sejm_session,date,political_club
8799,Poseł Adam Krzemiński,Panie Marszałku! Wysoka Izbo! Szanowna Pani Mi...,17,2024-09-12,KO


## Zapis dataframe do excela
plik z rozszerzeniem .xlsx

In [30]:
politicians_df.to_excel('politycy.xlsx', index=False)

# Operowanie na zescrappowanych danych

## Przygotowanie datasetu z excela
- Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv
- Oczyszczenie datasetu (wypowiedzi - utterance) z niepotrzebnych afixów "\r" "\n"

In [44]:
dataset_politycy = pd.read_csv('politycy.csv', encoding='utf-8', sep=';')
dataset_politycy = dataset_politycy.dropna(subset=["utterance", "politician", "sejm_session", "date", "political_club"])

In [45]:
dataset_politycy['utterance'] = dataset_politycy['utterance'].str.replace(r'[\n\r]', '', regex=True)

In [46]:
dataset_politycy.sample()

,politician,utterance,sejm_session,date,political_club
684,Poseł Zbigniew Bogucki,Szanowny Panie Marszałku! Wysoki Sejmie! Jest ...,1,2023-12-06,PiS


## Podzielenie wypowiedzi na mniejsze segmenty
- Wykonanie na każdym wierszu funkcji podzielenia i pofiltrowania
- Podział wypowiedzi na zdania, punkty graniczne to między innymi kropka, przecinek, znak zapytania itp.
- Pofiltrowanie aby w datasecie zostały TYLKO zdania zawierające więcej niż 10 wyrazów

In [55]:
import re

def split_and_filter(row):
    sentences = re.split(r'(?<=[.!?])\s+', row['utterance'])
    filtered_sentences = [s for s in sentences if len(s.split()) >= 10]
    return pd.DataFrame({'politician': row['politician'], 'utterance': filtered_sentences, 'political_club': row['political_club']})

dataset_filtered_politycy = pd.concat(dataset_politycy.apply(split_and_filter, axis=1).tolist(), ignore_index=True)

## Zapis dataframe do excela
plik z rozszerzeniem .xlsx

In [56]:
dataset_filtered_politycy.to_excel('politycy_filtered.xlsx', index=False)

# Model AI oraz embeddings

## Przygotowanie datasetu z excela
Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv

In [3]:
dataset_filtered_politycy = pd.read_csv('politycy_filtered.csv', encoding='utf-8', sep=';')
dataset_filtered_politycy = dataset_filtered_politycy.dropna(subset=["politician", "utterance", 'political_club'])

## Załadowanie odpowiedniego modelu AI oraz stworzenie funkcji
- funkcja get_embedding odpowiedzialna za stworzenie embeddingu na tekście
- funkcja get_similarity porównująca podobieństwo między embeddingami przy użyciu cosine similarity 

In [24]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [25]:
# Funkcja do konwersji tekstu na wektor (embedding)
def get_embedding(text):
    return model.encode(text)

def get_similarity(user_input, df_embeddings):
    # Wektor wypowiedzi użytkownika
    user_embedding = get_embedding(user_input)
    
    # Oblicz podobieństwo z każdą wypowiedzią w dataset
    similarities = []
    for index, row in df_embeddings.iterrows():
        politician_embedding = np.array(row['embedding'])
        
        similarity = cosine_similarity([user_embedding], [politician_embedding])[0][0]
        similarities.append((similarity, row['politician'], row['utterance']))

    # Posortuj podobieństwa
    similarities.sort(reverse=True, key=lambda x: x[0])
    
    top_n = 4  # Weź 4 najbliższe wypowiedzi
    avg_similarity = np.mean([sim for sim, _, _ in similarities[:top_n]])
    
    return avg_similarity, similarities[:top_n]

## Konwersja dataframe'u z wypowiedziami na embeddings
wykonanie "get_embedding" na każdej wypowiedzi z dataframe

In [6]:
from tqdm import tqdm

politycy_phrases_embeddings = [
    (get_embedding(row["utterance"]), row["politician"], row["utterance"], row['political_club']) 
    for _, row in tqdm(dataset_filtered_politycy.iterrows(), total=len(dataset_filtered_politycy))
]

100%|██████████| 164100/164100 [1:08:57<00:00, 39.67it/s]


In [28]:
df_politycy_phrases_embeddings = pd.DataFrame(politycy_phrases_embeddings, columns=["embedding", "politician", "utterance", 'political_club'])
print(df_politycy_phrases_embeddings.shape)

NameError: name 'politycy_phrases_embeddings' is not defined

In [29]:
data = []
for idx, row in df_politycy_phrases_embeddings.iterrows():
    embedding = row['embedding']  # Dostęp do kolumny 'embedding'
    politician = row['politician']
    utterance = row['utterance']
    political_club = row['political_club']
    
    data.append({
        "embedding": ','.join(map(str, embedding)),  # Zapis embeddingu jako string
        "politician": politician,
        "utterance": utterance,
        "political_club": political_club
    })

df_politycy_phrases_embeddings = pd.DataFrame(data)
df_politycy_phrases_embeddings.to_excel('politycy_phrases_embeddings.xlsx', index=False)

Zapisano embeddingi do pliku politycy_embeddings.xlsx!


In [30]:
df_politycy_phrases_embeddings.head(5)

,embedding,politician,utterance,political_club
0,"-0.08385012,0.34024927,0.05296642,-0.2829197,0...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,Szanowne Panie Posłanki i Szanowni Panowie Pos...,Nieznana partia
1,"0.06613037,0.33783805,0.32265285,0.02485717,0....",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Dziękuję, dziękuję i jeszcze raz dziękuję wszy...",Nieznana partia
2,"-0.01098752,0.08707595,-0.07281946,0.14722022,...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Oczywiście my, Polacy, w wielu sprawach istotn...",Nieznana partia
3,"0.0883656591,0.131972238,-0.27648738,0.0721738...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,Prowadzimy często gorące spory i zapewne będzi...,Nieznana partia
4,"-0.13960795,0.21945684,-0.30796283,-0.19085917...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Najważniejsze jednak jest to, że spory te rozs...",Nieznana partia


# Porównywanie wypowiedzi użytkownika z politykami

## Przygotowanie datasetu z excela
Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv oraz dokonanie konwersji na obiekt typu array na kolumnie "embedding"

In [2]:
df_politicians_embedded = pd.read_csv('politycy_phrases_embeddings.csv', encoding='utf-8', sep=';')
df_politicians_embedded = df_politicians_embedded.dropna(subset=["embedding", "politician", "utterance", 'political_club'])

In [4]:
df_politicians_embedded.sample()

,embedding,politician,utterance,political_club
154386,"0.09732806,-0.01931092,-0.30092317,0.09707727,...",Sekretarz Stanu w Ministerstwie Spraw Wewnętrz...,W ostatnich dniach otrzymaliśmy kwotę 55 mln e...,KO


In [3]:
df_politicians_embedded["embedding"] = df_politicians_embedded["embedding"].apply(lambda x: np.array(list(map(float, x.split(',')))))

In [4]:
print(f"Liczba wierszy: {len(df_politicians_embedded)}")
print(f"Kształt DataFrame: {df_politicians_embedded.shape}")
display(df_politicians_embedded.sample())

Liczba wierszy: 164100
Kształt DataFrame: (164100, 4)


,embedding,politician,utterance,political_club
104137,"[0.148475915, -0.0242511909, 0.159822509, -0.0...",Poseł Waldemar Andzel,"Modernizacja stopnia wodnego Rędzin na Odrze, ...",PiS


## Porównywanie

In [26]:
user_input = "Zielona energia jest bardzo istotna i powinniśmy skupić się na rozwoju rozwiązaniach ekologicznych. Musimy redukować energię węglową i zastapić ją energią z wiatru czy z wody."
#user_input = "Polacy potrzebują legalnej aborcji, ponieważ to jest prawo kobiety i nie można przymuszać kobiet."
#user_input = "Aborcja to zabójstwo dziecka i nie można tego zalegalizować. Jest to sprzeczne z jakimikolwiek prawami moralnymi i religijnymi"
#user_input = "Aborcja powinna być dozwolona w przypadku gwaltu, ciężkiej choroby dziecka lub możliwej śmierci matki. Ale nigdy nie dozwolona na życzenie."
#user_input = "Prawo do posiadania broni powinno być dozwolone dla przeciętnego obywatela. Ludzie muszą mieć możliwość obrony wlasnej."

avg_similarity, top_similarities = get_similarity(user_input, df_politycy_phrases_embeddings)

print(user_input)
print(f"Średnie podobieństwo: {avg_similarity}")
for similarity, politician, utterance in top_similarities:
    print(f"Polityk: {politician}, Wypowiedź: {utterance}, Podobieństwo: {similarity} \n")

Zielona energia jest bardzo istotna i powinniśmy skupić się na rozwoju rozwiązaniach ekologicznych. Musimy redukować energię węglową i zastapić ją energią z wiatru czy z wody.
Średnie podobieństwo: 0.7564250701458852
Polityk: Poseł Małgorzata Pępek, Wypowiedź: Ale wszyscy wiemy, że zielona energia potrzebuje przede wszystkim stabilności regulacyjnej., Podobieństwo: 0.7627143630860421 

Polityk: Poseł Tomasz Piotr Nowak, Wypowiedź: Otwieramy się na transformację energetyczną, otwieramy się na zieloną energię, ale także myślimy o stabilizacji systemu energetycznego, bo wiemy doskonale, że ta stabilizacja też musi być., Podobieństwo: 0.7616027139982423 

Polityk: Poseł Bożena Lisowska, Wypowiedź: Potrzeba nam strategii budowania większego potencjału zielonej energii i inwestycji w OZE, w rozwój technologii niskoemisyjnych, w tym technologii wodorowych, które pomogą nam uzyskać autonomię energetyczną., Podobieństwo: 0.7582319404508394 

Polityk: Poseł Urszula Pasławska, Wypowiedź: Przyspie

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import torch
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


In [8]:

# 3. Podział na zbiór treningowy i testowy (80% / 20%)
X = np.vstack(df_politicians_embedded['embedding'])  # Zbudowanie macierzy z embeddingów
y = df_politicians_embedded['political_club'].values  # Etykiety partii (kolumna 'political_club')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Trening modelu klasyfikacyjnego - np. regresja logistyczna
model = LogisticRegression(max_iter=500, multi_class='ovr')

with tqdm(total=X_train.shape[0], desc="Training Progress", unit="sample") as pbar:
    model.fit(X_train, y_train)
    pbar.update(X_train.shape[0])  # Gdy model zakończy trening, zaktualizuj postęp

# 5. Predykcja i ocena modelu
y_pred = model.predict(X_test)

# 6. Ocena modelu
print("Dokładność (Accuracy):", accuracy_score(y_test, y_pred))
print("\nRaport klasyfikacji:\n", classification_report(y_test, y_pred))


I:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
Training Progress: 100%|██████████| 131280/131280 [00:19<00:00, 6898.76sample/s]


Dokładność (Accuracy): 0.3723948811700183


I:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Raport klasyfikacji:
                  precision    recall  f1-score   support

             KO       0.35      0.54      0.42      9070
   Konfederacja       0.31      0.04      0.07      2291
         Lewica       0.38      0.05      0.09      2181
Nieznana partia       0.37      0.19      0.25      3022
         PSL-TD       0.38      0.06      0.10      2260
            PiS       0.39      0.67      0.50      9358
  Polska2050-TD       0.52      0.04      0.07      2787
          Razem       0.27      0.00      0.01       622
   Republikanie       0.33      0.01      0.02       772
          niez.       0.00      0.00      0.00       457

       accuracy                           0.37     32820
      macro avg       0.33      0.16      0.15     32820
   weighted avg       0.37      0.37      0.31     32820



I:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
I:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
# Załóżmy, że masz już dane: df_politicians_embedded
X = np.vstack(df_politicians_embedded['embedding'])
y = df_politicians_embedded['political_club'].values

# Podział na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trening modelu Random Forest
n_estimators = 20  # Liczba drzew w lesie
model = RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1, random_state=42)

# Ręczne monitorowanie postępu treningu
with tqdm(total=n_estimators, desc="Training Random Forest", unit="tree") as pbar:
    for _ in range(n_estimators):
        model.fit(X_train, y_train)  # Trenuj jeden model
        pbar.update(1)  # Zaktualizuj pasek po każdej iteracji

# Predykcja
y_pred = model.predict(X_test)

# Ocena modelu
print("Dokładność (Accuracy):", accuracy_score(y_test, y_pred))
print("\nRaport klasyfikacji:\n", classification_report(y_test, y_pred))


Training Random Forest: 100%|██████████| 20/20 [10:11<00:00, 30.58s/tree]


Dokładność (Accuracy): 0.32269957343083483

Raport klasyfikacji:
                  precision    recall  f1-score   support

             KO       0.31      0.55      0.40      9070
   Konfederacja       0.15      0.02      0.04      2291
         Lewica       0.16      0.03      0.05      2181
Nieznana partia       0.20      0.06      0.10      3022
         PSL-TD       0.27      0.04      0.07      2260
            PiS       0.36      0.54      0.43      9358
  Polska2050-TD       0.24      0.04      0.06      2787
          Razem       0.00      0.00      0.00       622
   Republikanie       0.20      0.00      0.00       772
          niez.       0.00      0.00      0.00       457

       accuracy                           0.32     32820
      macro avg       0.19      0.13      0.12     32820
   weighted avg       0.27      0.32      0.26     32820



In [11]:
party_counts = df_politicians_embedded['political_club'].value_counts()

print(party_counts)

political_club
PiS                47370
KO                 44674
Nieznana partia    15186
Polska2050-TD      13977
Konfederacja       11517
PSL-TD             11337
Lewica             11080
Republikanie        3756
Razem               2964
niez.               2239
Name: count, dtype: int64


In [17]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# 3. Podział na zbiór treningowy i testowy (80% / 20%)
X = np.vstack(df_politicians_embedded['embedding'])  # Zbudowanie macierzy z embeddingów
y = df_politicians_embedded['political_club'].values  # Etykiety partii (kolumna 'political_club')

# Kodowanie etykiet jako liczby
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 4. Trening modelu XGBoost
model = XGBClassifier(max_depth=6, n_estimators=50, use_label_encoder=False, eval_metric='mlogloss')

# Możemy użyć tqdm do śledzenia postępu, ale XGBoost sam obsługuje iteracje
with tqdm(total=X_train.shape[0], desc="Training XGBoost", unit="sample") as pbar:
    model.fit(X_train, y_train)
    pbar.update(X_train.shape[0])  # Gdy model zakończy trening, zaktualizuj postęp

# 5. Predykcja i ocena modelu
y_pred = model.predict(X_test)

# 6. Ocena modelu
print("Dokładność (Accuracy):", accuracy_score(y_test, y_pred))
print("\nRaport klasyfikacji:\n", classification_report(y_test, y_pred))

I:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [04:43:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Training XGBoost: 100%|██████████| 131280/131280 [01:48<00:00, 1208.59sample/s]

Dokładność (Accuracy): 0.3705971968312005

Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.35      0.55      0.43      9070
           1       0.25      0.04      0.07      2291
           2       0.34      0.05      0.09      2181
           3       0.33      0.16      0.21      3022
           4       0.39      0.07      0.12      2260
           5       0.40      0.66      0.50      9358
           6       0.34      0.04      0.08      2787
           7       0.30      0.01      0.02       622
           8       0.21      0.01      0.01       772
           9       0.00      0.00      0.00       457

    accuracy                           0.37     32820
   macro avg       0.29      0.16      0.15     32820
weighted avg       0.35      0.37      0.31     32820



In [18]:
!pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [21]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler

# Podział na dane treningowe i testowe (80/20)
X = np.vstack(df_politicians_embedded['embedding'])  # Zbudowanie macierzy z embeddingów
y = df_politicians_embedded['political_club'].values  # Etykiety partii (kolumna 'political_club')

# Kodowanie etykiet jako liczby
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Zastosowanie undersamplingu, aby zredukować liczbę próbek w większych klasach
undersampler = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = undersampler.fit_resample(X_train, y_train)

# 4. Trening modelu XGBoost
model = XGBClassifier(max_depth=6, n_estimators=50, use_label_encoder=False, eval_metric='mlogloss')

# Możemy użyć tqdm do śledzenia postępu, ale XGBoost sam obsługuje iteracje
with tqdm(total=X_train_res.shape[0], desc="Training XGBoost", unit="sample") as pbar:
    model.fit(X_train_res, y_train_res)
    pbar.update(X_train_res.shape[0])  # Gdy model zakończy trening, zaktualizuj postęp

# 5. Predykcja i ocena modelu
y_pred = model.predict(X_test)

# 6. Ocena modelu
print("Dokładność (Accuracy):", accuracy_score(y_test, y_pred))
print("\nRaport klasyfikacji:\n", classification_report(y_test, y_pred))

I:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [04:51:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Training XGBoost: 100%|██████████| 17820/17820 [00:32<00:00, 548.39sample/s]


Dokładność (Accuracy): 0.1765386959171237

Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.37      0.13      0.19      9070
           1       0.14      0.18      0.15      2291
           2       0.13      0.19      0.15      2181
           3       0.22      0.30      0.25      3022
           4       0.15      0.22      0.18      2260
           5       0.43      0.16      0.23      9358
           6       0.14      0.16      0.15      2787
           7       0.06      0.26      0.10       622
           8       0.06      0.28      0.10       772
           9       0.03      0.25      0.06       457

    accuracy                           0.18     32820
   macro avg       0.17      0.21      0.16     32820
weighted avg       0.29      0.18      0.19     32820



In [25]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from tqdm import tqdm

# Załóżmy, że X i y to dane i etykiety
X = np.vstack(df_politicians_embedded['embedding'])  # Macierz embeddingów
y = df_politicians_embedded['political_club'].values  # Etykiety klas

# Kodowanie etykiet jako liczby (jeśli etykiety są tekstowe)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Zastosowanie SMOTE (oversampling mniejszych klas)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Inicjalizacja modelu XGBoost
model = XGBClassifier(max_depth=6, n_estimators=50, use_label_encoder=False, eval_metric='mlogloss')

# Trening modelu XGBoost na zbalansowanym zbiorze
with tqdm(total=X_train_res.shape[0], desc="Training XGBoost", unit="sample") as pbar:
    model.fit(X_train_res, y_train_res)
    pbar.update(X_train_res.shape[0])

# Predykcja na zbiorze testowym
y_pred = model.predict(X_test)

# Ocena modelu
print("Dokładność (Accuracy):", accuracy_score(y_test, y_pred))
print("\nRaport klasyfikacji:\n", classification_report(y_test, y_pred))



I:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [04:57:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Training XGBoost: 100%|██████████| 380120/380120 [04:35<00:00, 1378.21sample/s]


Dokładność (Accuracy): 0.2626447288238879

Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.38      0.24      0.29      9070
           1       0.18      0.23      0.20      2291
           2       0.18      0.21      0.20      2181
           3       0.25      0.35      0.30      3022
           4       0.18      0.23      0.20      2260
           5       0.44      0.33      0.38      9358
           6       0.18      0.16      0.17      2787
           7       0.09      0.22      0.12       622
           8       0.08      0.23      0.12       772
           9       0.04      0.12      0.06       457

    accuracy                           0.26     32820
   macro avg       0.20      0.23      0.20     32820
weighted avg       0.31      0.26      0.28     32820

